# Data Exploration - Analyis of Infaltion Rates Data

In [20]:
import pandas as pd

Source: https://data-explorer.oecd.org/vis?tm=inflation&pg=0&snb=50&vw=tb&df%5Bds%5D=dsDisseminateFinalDMZ&df%5Bid%5D=DSD_PRICES%40DF_PRICES_HICP&df%5Bag%5D=OECD.SDD.TPS&df%5Bvs%5D=1.0&dq=HRV%2BBGR%2BTUR%2BGBR%2BCHE%2BSVN%2BSWE%2BESP%2BSVK%2BPRT%2BPOL%2BNOR%2BNLD%2BLUX%2BLTU%2BLVA%2BIRL%2BITA%2BISL%2BHUN%2BGRC%2BDEU%2BFRA%2BFIN%2BEST%2BDNK%2BBEL%2BCZE%2BAUT.M.HICP.CPI.PA._T.N.GY&to%5BTIME_PERIOD%5D=false&pd=2000-01%2C2024-10

In [21]:
df = pd.read_csv("../data/external/oecd-european-inflation-rates-since-2000.csv")
df.head()

,STRUCTURE,STRUCTURE_ID,STRUCTURE_NAME,ACTION,REF_AREA,Reference area,FREQ,Frequency of observation,METHODOLOGY,Methodology,...,OBS_STATUS,Observation status,UNIT_MULT,Unit multiplier,BASE_PER,Base period,DURABILITY,Durability,DECIMALS,Decimals
0,DATAFLOW,OECD.SDD.TPS:DSD_PRICES@DF_PRICES_HICP(1.0),Eurostat harmonised index of consumer prices (...,I,SVN,Slovenia,M,Monthly,HICP,Eurostat harmonised index of consumer prices (...,...,A,Normal value,NaN,NaN,NaN,NaN,NaN,NaN,2,Two
1,DATAFLOW,OECD.SDD.TPS:DSD_PRICES@DF_PRICES_HICP(1.0),Eurostat harmonised index of consumer prices (...,I,SVN,Slovenia,M,Monthly,HICP,Eurostat harmonised index of consumer prices (...,...,A,Normal value,NaN,NaN,NaN,NaN,NaN,NaN,2,Two
2,DATAFLOW,OECD.SDD.TPS:DSD_PRICES@DF_PRICES_HICP(1.0),Eurostat harmonised index of consumer prices (...,I,SVN,Slovenia,M,Monthly,HICP,Eurostat harmonised index of consumer prices (...,...,A,Normal value,NaN,NaN,NaN,NaN,NaN,NaN,2,Two
3,DATAFLOW,OECD.SDD.TPS:DSD_PRICES@DF_PRICES_HICP(1.0),Eurostat harmonised index of consumer prices (...,I,SVN,Slovenia,M,Monthly,HICP,Eurostat harmonised index of consumer prices (...,...,A,Normal value,NaN,NaN,NaN,NaN,NaN,NaN,2,Two
4,DATAFLOW,OECD.SDD.TPS:DSD_PRICES@DF_PRICES_HICP(1.0),Eurostat harmonised index of consumer prices (...,I,SVN,Slovenia,M,Monthly,HICP,Eurostat harmonised index of consumer prices (...,...,A,Normal value,NaN,NaN,NaN,NaN,NaN,NaN,2,Two


How many countires do we have data for?

In [22]:
df = df[["Reference area", "TIME_PERIOD", "OBS_VALUE"]]
df = df[df["Reference area"] != "United States"]
print(len(df.drop_duplicates(subset=["Reference area"])))
df_countries = df.drop_duplicates(subset=["Reference area"])["Reference area"]

29


Since when do we have data? Is the data complete for all countries?

In [23]:
data_points_per_country = {}
for country in df_countries.tolist():
    df_country = df[df["Reference area"] == country]
    data_points_per_country[country] = len(df_country)

data_points_per_country = dict(sorted(data_points_per_country.items(), key=lambda item: item[1]))
data_points_per_country

{'Switzerland': 226,
 'Denmark': 297,
 'Sweden': 297,
 'Poland': 297,
 'Norway': 297,
 'Hungary': 297,
 'United Kingdom': 297,
 'Iceland': 297,
 'Czechia': 297,
 'Türkiye': 297,
 'Bulgaria': 297,
 'Slovenia': 298,
 'Estonia': 298,
 'Belgium': 298,
 'Luxembourg': 298,
 'Lithuania': 298,
 'Netherlands': 298,
 'Austria': 298,
 'Ireland': 298,
 'Slovak Republic': 298,
 'Latvia': 298,
 'Italy': 298,
 'Greece': 298,
 'France': 298,
 'Germany': 298,
 'Finland': 298,
 'Spain': 298,
 'Portugal': 298,
 'Croatia': 298}

In [24]:
def get_min_max_date(df, country):
    df_country = df[df["Reference area"] == country]
    return (df_country["TIME_PERIOD"].min(), df_country["TIME_PERIOD"].max())

print(get_min_max_date(df, "Switzerland"))
print(get_min_max_date(df, "Denmark"))
print(get_min_max_date(df, "Germany"))


('2005-12', '2024-09')
('2000-01', '2024-09')
('2000-01', '2024-10')


As we can see, we have less data points for Switzerland (only since December 2005). We will handle this by removing all time periods for which we have missing data, as 226 data points should still be enough for our use case. This step will be done in a later stage.